In [1]:
from statsmodels.tsa.arima.model import ARIMA
import pandas as pd

In [2]:
average_RRC = pd.read_csv('sample-folder/RRC_Connected User-1.csv')

In [3]:
list1 = average_RRC.columns[0:]
result = pd.DataFrame()

In [4]:
average_RRC

,SECTOR-ID,2023-W01,2023-W02,2023-W03,2023-W04,2023-W05,2023-W06,2023-W07,2023-W08,2023-W09,...,2023-W25,2023-W26,2023-W27,2023-W28,2023-W29,2023-W30,2023-W31,2023-W32,2023-W33,2023-W34
0,11CLG0002-1,338.98,398.83,358.14,358.58,333.31,282.21,325.61,331.84,339.37,...,360.19,367.29,402.98,400.68,428.64,437.17,515.57,523.00,467.41,336.90
1,11CLG0002-2,146.07,133.33,153.47,148.83,131.88,116.62,140.44,145.35,143.66,...,135.65,148.02,141.90,140.44,146.44,155.65,137.60,143.98,133.59,112.29
2,11CLG0002-3,378.63,396.27,405.41,404.51,360.27,304.96,352.52,339.03,355.34,...,368.55,370.62,472.70,382.74,387.80,357.86,397.76,387.93,376.16,265.00
3,11CLG0003-1,218.11,205.29,191.24,171.71,173.37,177.58,176.11,182.65,177.29,...,167.40,196.07,230.85,226.03,194.12,176.47,160.35,166.52,148.18,134.33
4,11CLG0003-2,177.51,198.84,190.15,190.57,261.04,184.53,186.85,158.34,157.76,...,180.46,173.77,269.83,236.18,177.25,170.08,170.80,166.78,156.66,153.12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,11PDG0039-2,53.56,45.04,50.34,51.92,46.73,48.04,52.57,47.89,44.16,...,48.57,52.93,62.04,57.40,47.19,49.73,53.30,58.14,66.61,50.51
995,11PDG0039-3,68.31,69.15,70.93,80.64,75.26,69.41,71.29,71.14,72.73,...,63.34,87.10,83.38,90.64,86.34,68.79,62.92,67.69,65.84,63.87
996,11PDG0040-1,49.62,43.52,40.67,45.03,37.85,37.88,36.25,38.15,40.69,...,39.69,44.91,42.58,39.65,40.42,39.63,35.39,37.78,37.12,35.75
997,11PDG0040-2,76.43,61.84,53.62,61.53,55.92,54.65,55.68,56.39,54.14,...,57.18,69.33,63.46,58.41,67.30,57.33,61.84,63.80,63.35,54.97


In [5]:
for x in range(2, len(average_RRC.columns), 1):

    sitelist = average_RRC[['SECTOR-ID']]
    duration_week = pd.DataFrame(data={'date': [list1[x]] * len(average_RRC.index)})
    content = pd.DataFrame(average_RRC.iloc[:,x]).reset_index(drop=True)
    content.rename({content.columns[0]:'RRC'}, axis=1, inplace=True)

    concat = pd.concat([sitelist, duration_week, content], axis=1)
    result = pd.concat([result, concat], axis=0)

In [6]:
result.reset_index(inplace=True, drop=True)
result['Year'] = result['date'].str.split('-W').str[0]
result['WeekNumber'] = result['date'].str.split('-W').str[1].astype(int)

result['StartOfWeek'] = pd.to_datetime(result['Year'], format='%Y') + pd.to_timedelta((result['WeekNumber'] - 1) * 7 + 1, unit='D')
result.drop(columns=['Year', 'WeekNumber'], inplace=True)

result.rename(columns={'RRC': 0, 'StartOfWeek':'ds'}, inplace=True)

In [7]:
# concatenate = pd.DataFrame()

# query = result[result['SECTOR-ID'] == '11CLG0015-3'][['ds', 0]]

# query.set_index('ds', inplace=True)

# test = query
# test.index = test.index.to_period('W')
# model = ARIMA(test, order=(17,0,1))
# model = model.fit()
# forecast = model.forecast(steps=70)

# query = pd.concat([query, forecast], axis=0)
# query.reset_index(inplace=True)
# combine = pd.concat([query, pd.DataFrame({'SECTOR-ID':['11CLG0002-2']*len(query.index)})], axis=1)

# concatenate = pd.concat([concatenate, combine], axis=1)
# # concatenate.reset_index(inplace=True, drop=True)

In [8]:
# concatenate

In [9]:
concatenate = pd.DataFrame()

for x in result['SECTOR-ID'].unique().tolist():
    query = result[result['SECTOR-ID'] == x][['ds', 0]]
    query.set_index('ds', inplace=True)

    test = query
    test.index = test.index.to_period('W')
    model = ARIMA(test, order=(17,0,1), enforce_stationarity=False)
    model = model.fit()
    #make forecast for the next 69 weeks
    forecast = model.forecast(steps=70)

    forecast = pd.concat([query, forecast], axis=0)
    forecast.reset_index(inplace=True)

    # query
    combine = pd.concat([forecast, pd.DataFrame({'SECTOR-ID':[x]*len(forecast.index)})], axis=1)

    concatenate = pd.concat([concatenate, combine], ignore_index=True, axis=0)

/home/nivag/2023-Linux/.python-3.10/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/home/nivag/2023-Linux/.python-3.10/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/home/nivag/2023-Linux/.python-3.10/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/home/nivag/2023-Linux/.python-3.10/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/home/nivag/

KeyboardInterrupt: 

In [10]:
concatenate

,index,0,SECTOR-ID
0,2023-01-09/2023-01-15,398.830000,11CLG0002-1
1,2023-01-16/2023-01-22,358.140000,11CLG0002-1
2,2023-01-23/2023-01-29,358.580000,11CLG0002-1
3,2023-01-30/2023-02-05,333.310000,11CLG0002-1
4,2023-02-06/2023-02-12,282.210000,11CLG0002-1
...,...,...,...
819,2024-11-25/2024-12-01,-69440.476790,11CLG0004-2
820,2024-12-02/2024-12-08,-2283.058981,11CLG0004-2
821,2024-12-09/2024-12-15,-46787.311966,11CLG0004-2
822,2024-12-16/2024-12-22,-54896.646273,11CLG0004-2
